### Unidad 2: Taller de análisis de datos textuales

<h1> Notebook 3 - Introducción a la vectorización de palabras: Word2Vec</h1>

En los notebooks anteriores, hemos hablado de los vectores que se utilizan para representar nuestros datos, textuales o no, en una forma matemática, y así poder aplicar métodos de machine learning.

En este notebook, vamos a llevar esta idea un paso más allá y generar reprentaciones vectoriales de las <u>palabras</u>. Esta idea es generalmente llamada __Word Embeddings__. __Word2Vec__ es un algoritmo popular para implementar esta idea.
Esta técnica permite para representar mejor el significado de una palabra.

Para más detalles sobre los fundamentos de Word2Vec, se puede leer: _Efficient Estimation of Word Representations in Vector Space_ [[Mikolov et al., 2013]](https://arxiv.org/pdf/1301.3781.pdf)

## 1. ¿Qué es una representación vectorial de una palabra?

Supongamos que nuestro vocabulario tiene sólo cinco palabras: King, Queen, Man, Woman y Child. Podríamos codificar la palabra 'Queen' como:

<img src="word2vec1.png"/>

Usando tal codificación, no hay comparación interesante que podamos hacer entre vectores de palabras. Con word2vec, se utiliza una representación distribuida de una palabra. Cada palabra está representada por una distribución de pesos entre esos elementos. La representación de una palabra se extiende a través de todos los elementos en el vector, y cada elemento en el vector contribuye a la definición de muchas palabras.

<img src="word2vec2.png"/>

Tal vector llega a representar de alguna manera abstracta el 'significado' de una palabra. Y como veremos a continuación, simplemente examinando un corpus grande es posible aprender vectores de palabras que son capaces de capturar las relaciones entre palabras de una manera sorprendentemente expresiva. 


### Razonamiento con vectores de palabras

Los vectores son muy buenos para responder a problemas de analogía de tipo: '_A_ es a _B_ lo que _C_ es a ...?'. Por ejemplo, el _hombre_ es a la _mujer_ lo que el _tío_ es a la _tía_ usando un método de desplazamiento vectorial.

Por ejemplo, el desplazamiento vectorial que ilustra la _relación de género_:

<img src="word2vec3.png"/>


Este tipo de composición vectorial también nos permite responder a la pregunta __King - Man + Woman = ?__ y llegar al resultado __Queen__.

<img src="word2vec4.png"/>

Estos modelos vectoriales de palabras están destacables ya que permiten representar ciertas relaciones entre palabras sin aportar información sobre la semántica de las palabras. Como vamos a verlo, se pueden aprender los vectores a partir de gran conjuntos de datos textuales.

Algunos ejemples de relaciones entre palabras que se han podido calcular utilizando Word2Vec:

<img src="word2vec6.png"/>

<img src="word2vec7.png"/>

También podemos utilizar la adición de vectores para hacer preguntas como "German + airlines" y mirando las palabras más cercanas al vector compuesto obtenemos respuestas impresionantes:

<img src="word2vec8.png"/>

### Construir una representación vectorial de una palabra _y_ considerando las palabras X que aparacen frecuentemente en su contexto

Para construir sus vectores, Word2Vec utiliza un dataset de entrenamiento y algoritmos de aprendizaje basados en redes neuronales (__Continuous Bag of Words__ (CBOW), o modelo __Skip Gram__). El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' si aparece la palabra 'pelota' en el contexto?

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>

__Algoritmo CBOW__

Las palabras de contexto forman la capa de entrada. Si el tamaño del vocabulario es V, estos serán vectores de dimensión V con sólo uno de los elementos establecido en uno, y el resto todos los ceros. Hay una sola capa oculta y una capa de salida.

<img src="word2vec9.png"/>

## 2. Word2Vec: crear representaciones vectoriales de palabras

La clase <code>word2vec</code> de Gensim permite entrenar modelos vectoriales de palabras (ver documentación: https://radimrehurek.com/gensim/models/word2vec.html).

Esta clase tiene varios parametros, en particular:
- <code>sentences</code>: una lista de palabras o de frases que sirve para entrenar el modelo
- <code>sg</code>: define que algoritmos de aprendizaje utilizar (0=CBOW, 1=skip-gram)
- <code>size</code>: define la dimensión de los vectores que se desea extraer
- <code>window</code>: define el número de palabras considerar a la izquierda y a la derecha de una palabra
- <code>min_count</code>: ignorar las palabras que aparecen menos de _min_count_
y otros asociados a la parametrización de la fase de aprendizaje de la red neuronal (que no detallaremos en esta parte del curso):
- <code>alpha</code>: el _learning rate_ utilizado para optimizar los parametros de la red neuronal.
- <code>iter</code>: número de iteraciones (epocas) sobre el dataset para encontrar los parametreos que optimizan la red neuronal.

In [2]:
from gensim.models import word2vec

Para entrenar nuestro modelo Word2Vec, podemos utilizar nuestros propios datasets o utilizar datasets genericos existentes. Para empezar, utilizaremos 100 MB de textos extraidos de Wikipedia en inglés, para generar vectores de 200 dimensiones.

In [3]:
sentences = word2vec.Text8Corpus('text8.txt')

In [4]:
model = word2vec.Word2Vec(sentences,size=200,hs=1)

In [4]:
print(model)

Word2Vec(vocab=71290, size=200, alpha=0.025)


Ahora que hemos aprendido nuestro modelo, tratemos de resolver la ecuación <code>King - Man + Woman</code>.

En otras palabras buscamos cuál es el vector más similar al vector que adiciona positivamente 'King' y 'Woman' y negativamente 'Man'.

In [5]:
model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=5)

[('queen', 0.5421801209449768),
 ('throne', 0.535997211933136),
 ('aragon', 0.5014480352401733),
 ('prince', 0.496055543422699),
 ('regent', 0.49364346265792847)]

In [6]:
model.wv.most_similar(positive=["conflict"])

[('conflicts', 0.7148991823196411),
 ('clashes', 0.678907036781311),
 ('confrontation', 0.6748725771903992),
 ('hostilities', 0.6699436902999878),
 ('tensions', 0.6349812746047974),
 ('struggle', 0.6328939199447632),
 ('confrontations', 0.6149111986160278),
 ('strife', 0.6127475500106812),
 ('dispute', 0.5990400314331055),
 ('struggles', 0.5930981636047363)]

In [7]:
model.wv.most_similar(positive=["conflict","weapon"])

[('confrontation', 0.590645432472229),
 ('fighting', 0.5415654182434082),
 ('weapons', 0.5375379323959351),
 ('warfare', 0.5332213640213013),
 ('hostilities', 0.51864093542099),
 ('clashes', 0.512557864189148),
 ('conflicts', 0.5101333260536194),
 ('struggle', 0.5092238783836365),
 ('melee', 0.5075847506523132),
 ('pistol', 0.49839386343955994)]

In [8]:
model.wv.most_similar(positive=["conflict"],negative=["weapon"])

[('conflicts', 0.5014094114303589),
 ('disagreements', 0.48494234681129456),
 ('dispute', 0.448714941740036),
 ('clashes', 0.445597767829895),
 ('disputes', 0.4424167275428772),
 ('frictions', 0.4384433329105377),
 ('tensions', 0.43828195333480835),
 ('controversies', 0.4309636950492859),
 ('hostilities', 0.4257396459579468),
 ('confrontations', 0.41652169823646545)]

In [9]:
model.wv.most_similar(positive=["life"])

[('lives', 0.5042439699172974),
 ('childhood', 0.4872216284275055),
 ('career', 0.47070246934890747),
 ('humanity', 0.4319986402988434),
 ('teens', 0.42829176783561707),
 ('work', 0.42276912927627563),
 ('experiences', 0.4157204031944275),
 ('universe', 0.4052744209766388),
 ('genius', 0.3996080756187439),
 ('intellect', 0.39646589756011963)]

In [10]:
model.wv.most_similar(positive=["life","love"])

[('passion', 0.5265476703643799),
 ('happiness', 0.5208061933517456),
 ('affection', 0.5124889612197876),
 ('dreams', 0.4949033856391907),
 ('humanity', 0.4934290051460266),
 ('sadness', 0.4914532005786896),
 ('mercy', 0.48590323328971863),
 ('me', 0.4846687912940979),
 ('lover', 0.4835691452026367),
 ('passionate', 0.48191386461257935)]

In [11]:
model.wv.most_similar(positive=["life","money"])

[('profits', 0.5068252086639404),
 ('fortune', 0.49457332491874695),
 ('debts', 0.4487438201904297),
 ('estate', 0.4476965665817261),
 ('happiness', 0.4473707675933838),
 ('wealth', 0.44621741771698),
 ('work', 0.43607115745544434),
 ('goods', 0.42565661668777466),
 ('satisfaction', 0.42506659030914307),
 ('donations', 0.4217737317085266)]

In [12]:
model.wv.most_similar(positive=["life","happiness"])

[('humanity', 0.588325560092926),
 ('dignity', 0.5490078330039978),
 ('mankind', 0.5261237025260925),
 ('intellect', 0.5199679732322693),
 ('salvation', 0.504727840423584),
 ('lives', 0.5021054148674011),
 ('compassion', 0.4933162331581116),
 ('truth', 0.4921111464500427),
 ('feelings', 0.4915164113044739),
 ('experience', 0.49044179916381836)]

In [13]:
model.wv.most_similar(positive=["life"],negative=["health"])

[('universe', 0.3619164824485779),
 ('depiction', 0.35000133514404297),
 ('narrative', 0.3393925428390503),
 ('prose', 0.3365318179130554),
 ('genius', 0.3228011727333069),
 ('circle', 0.30867573618888855),
 ('poem', 0.301101416349411),
 ('romances', 0.29319894313812256),
 ('retelling', 0.292746901512146),
 ('vivid', 0.28444328904151917)]

Ver los parametros aprendidos por la red neuronal para una palabra dada:

In [14]:
model.wv['computer']

array([ 1.45870554e+00,  3.19150418e-01,  6.78649545e-02,  4.17201877e-01,
       -9.46381509e-01,  1.36610970e-01,  6.96709931e-01, -4.09129471e-01,
       -2.60210800e+00, -5.74939966e-01,  6.19564831e-01, -1.27176726e+00,
        4.79786247e-01, -1.75121868e+00, -1.17371380e+00, -8.00383806e-01,
       -1.03532290e+00,  6.73200965e-01, -1.36196721e+00, -2.01697499e-01,
       -6.89549565e-01, -1.30584931e+00, -1.58035070e-01,  2.15400219e+00,
       -8.64808321e-01, -1.55891311e+00,  6.76406801e-01,  1.37176073e+00,
       -9.23945725e-01, -3.53786230e-01, -3.92308831e-01, -4.20996487e-01,
       -8.79794538e-01,  6.13480389e-01, -1.35208225e+00, -7.05701768e-01,
       -1.63380647e+00,  3.27699780e-01, -3.51137459e-01, -1.51311362e+00,
       -1.37475801e+00, -6.20688438e-01,  4.92847025e-01,  7.17741668e-01,
       -8.35705996e-01,  1.28982675e+00, -2.65268475e-01,  5.99615602e-03,
       -9.72501814e-01, -3.56001228e-01, -7.44042277e-01, -1.77630830e+00,
       -3.43044668e-01, -

In [15]:
model.save("text8_model")
model=word2vec.Word2Vec.load("text8_model")

Otro ejemplo de aplicación: __buscar el vector más diferente de los otros__.

Entre las palabras siguientes, ¿cuál es la palabra la más distinta?

In [16]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'cereal'

In [17]:
model.wv.doesnt_match("brazil chile france peru argentina".split())

'france'

In [20]:
model.wv.doesnt_match("apple pear banana hammer".split())

'apple'

Podemos tambien medir la similaridad entre palabras:

In [21]:
model.wv.similarity('man','woman')

0.6551958

In [22]:
model.wv.similarity('man','hammer')

0.21070464

In [23]:
model.wv.similarity('woman','hammer')

0.10536292

In [24]:
model.wv.similarity('man','engineer')

0.07649841

In [25]:
model.wv.similarity('woman','engineer')

0.06109159

In [26]:
model.wv.similarity('man','baby')

0.3630408

In [27]:
model.wv.similarity('woman','baby')

0.45751432

El modelo aprendido con la red neuronal captura el significado de las palabras, pero también los sesgos humanos.


Dado que los vectores obtenidos representan mejor el sentido de las palabras que las palabras mismas, sería interesante representar los documentos (textos, moticias) utilizando estos vectores. Es la idea del algoritmo __Doc2Vec__.

## 3. Doc2Vec: Representación vectorial de los documentos utilizando _Word embeddings_

Sabemos lo importante que es la representación vectorial de los documentos, en todo tipo de tareas, por ejemplo de clasificación supervisada (_sentiment analysis_, etc.) o de _topic modeling_.

Por cada una de estas tareas, tenemos que representar nuestros documentos como vectores. Hasta ahora, hemos visto representaciones vectoriales simples basados en modelos Bag-of-Words (BOW) o TF-IDF.

Basándose en Word2Vec, los investigadores en informática también han implementado en los últimos años una representación vectorial de documentos, popularmente llamada __Doc2Vec__.	Esto significa que ahora podemos usar el poder de la comprensión semántica de Word2Vec para describir documentos también.

Para una presentación del funcionamiento interno de Doc2Vec, se puede leer el artículo de blog siguiente: https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

En nuestro caso, por el momento, utilizaremos Doc2Vec como una caja negra para generar representaciones vectoriales de los documentos.

In [28]:
import gensim
import os
import collections
import smart_open
import random

Para ponernos en marcha, necesitaremos un conjunto de documentos para entrenar a nuestro modelo doc2vec. En teoría, un documento puede ser cualquier cosa, desde un breve tweet de 140 caracteres, un solo párrafo (por ejemplo, un resumen de un artículo de una revista), un artículo de noticias o un libro. En Tratamiento Automáticamente del Lenguaje, una colección o conjunto de documentos a menudo se denomina corpus.

Para este tutorial, vamos a entrenar nuestro modelo usando el Lee Background Corpus incluido en gensim. Este corpus contiene 314 documentos seleccionados del servicio de correo de noticias de la Corporación Australiana de Radiodifusión, que proporciona correos electrónicos de texto con titulares y cubre una serie de temas generales.

Y probaremos nuestro modelo a ojo usando una muestra específica del Lee Corpus mucho más corto que contiene 50 documentos.

In [29]:
test_data_dir ='{}'.format(os.sep).join([gensim.__path__[0],'test','test_data'])
lee_train_file=test_data_dir+os.sep+'lee_background.cor'
lee_test_file=test_data_dir+os.sep+'lee.cor'

__Definir una función para leer y preprocesar el texto__

A continuación, definimos una función para abrir el archivo de training y test (con codificación latina), leer el archivo línea por línea, preprocesar cada línea usando una simple herramienta de preprocesamiento de gensim (es decir, tokenizar texto en palabras individuales, eliminar la puntuación, ponerlo en minúsculas, etc.), y devolver una lista de palabras.

Tenga en cuenta que, para un archivo determinado, cada línea constituye un único documento y la longitud de cada línea (es decir, documento) puede variar. Además, para entrenar el modelo, necesitaremos asociar una etiqueta/número a cada documento del corpus de entrenamiento. En nuestro caso, la etiqueta es simplemente el número de línea basado en cero.

In [30]:
def read_corpus(file_name,tokens_only=False):
    with smart_open.smart_open(file_name,encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line),[i])

In [31]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file,tokens_only=True))

Miremos el resultado del preprocesamiento, observando los 2 primeros documentos del corpus de entrenamiento:

In [32]:
train_corpus[:2]

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

y el dataset de test:

In [33]:
test_corpus[:2]

[['the',
  'national',
  'executive',
  'of',
  'the',
  'strife',
  'torn',
  'democrats',
  'last',
  'night',
  'appointed',
  'little',
  'known',
  'west',
  'australian',
  'senator',
  'brian',
  'greig',
  'as',
  'interim',
  'leader',
  'shock',
  'move',
  'likely',
  'to',
  'provoke',
  'further',
  'conflict',
  'between',
  'the',
  'party',
  'senators',
  'and',
  'its',
  'organisation',
  'in',
  'move',
  'to',
  'reassert',
  'control',
  'over',
  'the',
  'party',
  'seven',
  'senators',
  'the',
  'national',
  'executive',
  'last',
  'night',
  'rejected',
  'aden',
  'ridgeway',
  'bid',
  'to',
  'become',
  'interim',
  'leader',
  'in',
  'favour',
  'of',
  'senator',
  'greig',
  'supporter',
  'of',
  'deposed',
  'leader',
  'natasha',
  'stott',
  'despoja',
  'and',
  'an',
  'outspoken',
  'gay',
  'rights',
  'activist'],
 ['cash',
  'strapped',
  'financial',
  'services',
  'group',
  'amp',
  'has',
  'shelved',
  'million',
  'plan',
  'to',
 

__Entrenamiento del modelo: Instanciación de un objeto Doc2Vec__

Ahora, instanciaremos un modelo de Doc2Vec con un tamaño vectorial de 50 palabras e iterando sobre el corpus de entrenamiento 40 veces. Fijamos el número mínimo de palabras en 2 para poder descartar palabras con muy pocas ocurrencias (sin una variedad de ejemplos representativos, retener palabras tan poco frecuentes puede empeorar el modelo).

Noten que se trata de un conjunto de datos muy pequeño (300 documentos) con documentos más cortos (unos pocos cientos de palabras).

In [34]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=40)

/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:576: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


__Construir el vocabulario__

In [35]:
model.build_vocab(train_corpus)

Tipicamente, el vocabulario es un diccionario (accesible a través de <code>model.wv.vocabulario</code>) de todas las palabras únicas extraídas del corpus de entrenamiento junto con el recuento.

Por ejemplo, model.wv.vocab['penalty'].count para recuentos para la palabra 'penalty'.

In [36]:
model.wv.vocab['penalty'].count

4

__Entrenar el modelo Doc2Vec__

In [37]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

/home/mvernier/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


__Inferir un vector__

Una cosa importante a tener en cuenta es que ahora puede inferir un vector para cualquier pieza de texto sin tener que volver a entrenar al modelo pasando una lista de palabras a la función model.infer_vector. Este vector puede entonces compararse con otros vectores a través de la similitud del coseno.


In [38]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([ 0.00656488,  0.00708244, -0.17919137,  0.04648336, -0.12993729,
        0.06353014,  0.19703127,  0.05113358,  0.09219772, -0.11792724,
       -0.03619491, -0.19111508, -0.08708404, -0.05263122,  0.22998478,
        0.16862209, -0.06500445, -0.05243946,  0.05479506, -0.02654412,
       -0.03320115,  0.06894251, -0.03920032,  0.01925326,  0.10807507,
       -0.17534779,  0.00671286,  0.03311221, -0.11576382,  0.18120219,
       -0.15539669,  0.25996867,  0.13454081, -0.04352678, -0.139891  ,
        0.07212743,  0.06612439, -0.11469728,  0.21276253, -0.30474105,
        0.13809398, -0.09311282, -0.06837962, -0.03343724, -0.1250036 ,
       -0.03129051, -0.1317404 ,  0.06516594, -0.08678626, -0.02467609],
      dtype=float32)

Tenga en cuenta que <code>infer_vector()</code> no toma una String, sino más bien una lista de tokens, que ya deberían haber sido tokenizados de la misma manera que las palabras propiedad de los objetos originales del documento de formación.

También tenga en cuenta que debido a que los algoritmos de entrenamiento son un problema de aproximación iterativa que hace uso de la aleatorización interna, las inferencias repetidas del mismo texto devolverán vectores ligeramente diferentes.

__Validación del modelo__

Para evaluar nuestro nuevo modelo, primero inferiremos nuevos vectores para cada documento del corpus de entrenamiento, compararemos los vectores inferidos con el corpus de entrenamiento, y luego devolveremos el rango del documento basado en la autosimilaridad. 

Básicamente, estamos suponiendo que el corpus de entrenamiento es un dato nuevo e invisible y luego vemos cómo se compara con el modelo entrenado. La expectativa es que probablemente hemos adaptado demasiado nuestro modelo (es decir, todos los rangos serán inferiores a 2) y por lo tanto deberíamos ser capaces de encontrar documentos similares muy fácilmente. Además, realizaremos un seguimiento de las segundas posiciones para comparar documentos menos similares.

In [39]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

Contemos cómo se clasifica cada documento con respecto al corpus de entrenamiento:

In [49]:
collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus

Counter({0: 292, 1: 8})

Básicamente, más del 95% de los documentos inferidos son más similares a sí mismos y alrededor del 5% de las veces es erróneamente más similar a otro documento. La comprobación de un vector inferido contra un vector entrenado es una especie de control sobre si el modelo se está comportando de una manera útil y coherente, aunque no un valor real de "exactitud".

Podemos echar un vistazo a un ejemplo:

In [50]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

Note arriba que el documento más similar tiene una puntuación de similitud que se acerca a 1.0. Sin embargo, la puntuación de similitud para los documentos de segundo orden debería ser significativamente menor.

Podemos ejecutar la siguiente celda repetidamente para ver una muestra de otras comparaciones de documento-objetivo.

In [52]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (272): «the storm clean up in sydney will resume in earnest this morning as fresh crews are brought in to replace state emergency service ses personnel who worked through the night the storm hit sydney early yesterday afternoon and two schoolgirls died when tree fell on them at reserve at hornsby heights in the city north number of other people were injured as the storm brought down trees and power poles and lifted roofs new south wales emergency services minister bob debus says welfare and emergency funding arrangements have been put in place with the declaration of natural disaster areas in campbeltown hornsby warringah and kurringai welfare services become available if they are needed local government is refunded any money it spends on the clean up or that it spends on repairing its own infrastructure low interest loans if they are needed are available to small business to help them get back on their feet again mr debus said energy australia says power has been restor

__Prueba del modelo__

Usando el mismo enfoque anterior, inferiremos el vector para un documento de prueba elegido al azar, y compararemos el documento con nuestro modelo a ojo.


In [53]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (8): «hunan province remained on high alert last night as thunderstorms threatened to exacerbate the flood crisis now entering its fifth day and with already dead and hundreds of thousands evacuated on the flood frontline at dongting lake the water level peaked at just under on saturday night then eased about cm during the day under hot sun with temperatures reaching but with the lake still brimming at dangerously high levels and spilling over the top of its banks in some places locals were fearful that thunderstorm and high winds forecast to hit the region last night would damage the dikes about km of dikes around the lake are all that stand between million people in the surrounding farmland and disaster»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,mc2,s0.001,t3):

MOST (77, 0.6922479867935181): «the woomera detention centre in outback south australia has experienced its first quiet night this week since sunday it seems the government decision to put freeze

## 4. Ejercicios - TP

1. Entrenar un modelo Word2Vec para el español. Se puede utilizar agrupar datasets de "Sophia" en un solo dataset grande: 

2. Entrenar un modelo Doc2Vec para el español.

3. Utilizar representaciones Doc2Vec para volver a resolver tareas de clasificación y/o de topic modeling.